## **Script Upload Google Sheets**

#### **Library dan Setting Connection**

In [1]:
# pip install gspread

In [2]:
# pip install --upgrade oauth2client 


In [3]:
import json
import gspread
from oauth2client.service_account import ServiceAccountCredentials

### ini adalah file key connection
with open('digitalskola_key.json','rb') as file:
    key = json.load(file)
    
scope = ['https://www.googleapis.com/auth/drive','https://spreadsheets.google.com/feeds']
creds = ServiceAccountCredentials.from_json_keyfile_dict(key, scope)
client = gspread.authorize(creds)

### tambahkan email googledigitalskola@digitalskola-368401.iam.gserviceaccount.com 
### ke dalam google sheet anda (seperti menambahkan atau memberikan hak akses kepada google sheet)#

## **Build Connection MongoDB**

In [4]:
import json
from pymongo import MongoClient

# Load configuration from the JSON file
with open('producer/config.json', 'r') as config_file:
    config = json.load(config_file)

# MongoDB connection details
mongodb_config = config['mongodb']
mongo_uri = mongodb_config['uri']
mongo_db = mongodb_config['db']

try:
    # Connect to MongoDB using the loaded URI
    client = MongoClient(mongo_uri)
    db = client[mongo_db]  # Select the database
    server_status = db.command("ping")
    print("MongoDB connection successful:", server_status)

    # List all databases
    databases = client.list_database_names()
    print("Databases:", databases)

except Exception as e:
    print("An error occurred:", e)


MongoDB connection successful: {'ok': 1.0}
Databases: ['admin', 'config', 'ftde02', 'ftde03', 'ftde3', 'local']


## **Load MongoDB Collection**

In [5]:
import json
from pymongo import MongoClient
import pandas as pd

try:
    # Load MongoDB configuration from JSON config
    with open('producer/config.json', 'r') as config_file:
        config = json.load(config_file)
    
    mongo_config = config['mongodb']
    mongo_uri = mongo_config['uri']

    # Connect to MongoDB server using the URI from config
    server = MongoClient(mongo_uri)
    
    # Verify the connection
    db = server.admin
    server_status = db.command("ping")
    print("MongoDB connection successful:", server_status)

    # Access a specific database
    database_name = "ftde3"  # Replace with your database name
    db = server[database_name]
    print(f"Using database: {database_name}")

    # Access a specific collection
    collection_name = "fraud_jeanneta"  # Replace with your collection name
    collection = db[collection_name]

    # Query the collection (fetch first 5 documents)
    documents = collection.find()

    # Convert documents to a DataFrame
    data_list = list(documents)  # Convert cursor to a list
    df = pd.DataFrame(data_list)  # Create a DataFrame from the list

    print("Data saved into DataFrame")

except Exception as e:
    print("An error occurred:", e)


MongoDB connection successful: {'ok': 1.0}
Using database: ftde3
Data saved into DataFrame


In [6]:
df

,_id,step,type,amount,nameOrig,newbalanceOrig,nameDest,newbalanceDest,oldbalanceOrg,oldbalanceDest,predict
0,676a0e53f7036fea14c7b262,1,PAYMENT,2734.77,C1756207614,1776.23,M1214133948,0.00,4511.00,0.00,White List
1,676a0e58f7036fea14c7b263,1,CASH_OUT,102153.86,C2015301874,0.00,C1335050193,353532.56,1776.23,230676.07,White List
2,676a0e5df7036fea14c7b264,1,PAYMENT,584.52,C712627377,14894.48,M782228073,0.00,15479.00,0.00,White List
3,676a0e62f7036fea14c7b265,1,PAYMENT,6113.14,C399373008,9515.86,M391506011,0.00,15629.00,0.00,White List
4,676a0e67f7036fea14c7b266,1,CASH_OUT,369989.20,C1936550492,0.00,C1789550256,4619798.56,9515.86,518243.06,White List
...,...,...,...,...,...,...,...,...,...,...,...
181,676a11ddf7036fea14c7b317,1,CASH_IN,110226.34,C1475192960,1601450.63,C1816757085,10681238.79,1601450.63,1601450.63,White List
182,676a11e2f7036fea14c7b318,1,CASH_IN,142577.44,C727197178,1744028.07,C1100439041,0.00,1601450.63,174480.01,White List
183,676a11e7f7036fea14c7b319,1,CASH_IN,6915.38,C1406253491,1750943.45,C85777802,0.00,1744028.07,49685.00,White List
184,676a11ecf7036fea14c7b31a,1,CASH_IN,157108.01,C749604930,1908051.46,C985934102,971418.91,1750943.45,1576572.37,White List


### **Uplaod Data dari DataFrame ke Google Sheet**

In [52]:
# ganti dengan nama google sheets anda
sheet = client.open('ftde03-jeanneta')

# ganti sesuai dengan nama sheet didalam google sheets anda
# siapkan nama kolom pada sheet di google sheet anda
export = sheet.worksheet('fraud')
export.update([df.columns.values.tolist()] + df.astype(str).values.tolist())

{'spreadsheetId': '1AW94j_xIgFpAHG8MZeVdvj9wG3Gudjwcyavr8plMgLg',
 'updatedRange': 'fraud!A1:K1843',
 'updatedRows': 1843,
 'updatedColumns': 11,
 'updatedCells': 20273}